In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [3]:
X, y = data.data, data.target

In [6]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
n_samples, n_features = X.shape

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [14]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [15]:
print(y_train.shape)
print(y_test.shape)

torch.Size([455, 1])
torch.Size([114, 1])


In [17]:
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

In [18]:
num_epochs = 100
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch --> {epoch+1}, loss --> {loss.item():.2f}')

epoch --> 10, loss --> 0.53
epoch --> 20, loss --> 0.52
epoch --> 30, loss --> 0.51
epoch --> 40, loss --> 0.50
epoch --> 50, loss --> 0.49
epoch --> 60, loss --> 0.49
epoch --> 70, loss --> 0.48
epoch --> 80, loss --> 0.47
epoch --> 90, loss --> 0.47
epoch --> 100, loss --> 0.46


In [24]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.8333
